In [3]:
#Importing all the necessary libraries I would require for this project
from dbfread import DBF
import numpy as np
from skimage import data, io, filters

import tensorflow as tf
import os
import random

from tqdm import tqdm 
from zipfile import ZipFile

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from osgeo import gdal

In [6]:
#Reading the CDL map
I = plt.imread("/content/CDL_2013_Champaign_north.tif")

In [ ]:
#Reading the DBF to see the RGB values of corn and soybeans, so that I could create my labels.
#For this, I have not used the last dimension that is, opacity. Kept it as it is.
l = []
for record in DBF("D:/data/CDL_2013_clip_20170525181724_1012622514.tif.vat.dbf"):
    l.append(record)

In [7]:
#Creating labels from CDL Map. Mapped RGB values with [ 1., 0.82745,   0.] with 2 that is corn.
#Mapped RGB values with [0.14902, 0.43922, 0.] with 1 that is soybeans
#Kept 0 for all other cases.

#Scaled the input image by dividing with 255. to get in the range of values present in DBF file.
Iscaled = I/255.
#Removed Opacity
Iscaled = Iscaled[:,:,:3]
#Initializing zeros with shape (5959, 9425)
labels = np.zeros((Iscaled.shape[0], Iscaled.shape[1]), dtype=np.float)
Iscaled = np.round(Iscaled,5)
#Mapping of the values.
labels[np.all(Iscaled == np.array([0.14902, 0.43922, 0.        ]), axis=-1)]=1#soybeans
labels[np.all(Iscaled == np.array([ 1., 0.82745,   0.]), axis=-1)] = 2#Corn
#reshaped for the further use.
labels = labels.reshape(labels.shape[0], labels.shape[1], 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


In [13]:
#Reading the training image.
dataset = gdal.Open("/content/20130824_RE3_3A_Analytic_Champaign_north.tif")

In [14]:
#Reading the 5 bands present in the image.
red_channel = dataset.GetRasterBand(1).ReadAsArray()
green_channel = dataset.GetRasterBand(2).ReadAsArray()
blue_channel = dataset.GetRasterBand(3).ReadAsArray()
red_edge = dataset.GetRasterBand(4).ReadAsArray()
infrared_edge = dataset.GetRasterBand(5).ReadAsArray()

In [15]:
#Stacking all my final channels and creating a final image with shape (5959,9425,5)
img = np.dstack((red_channel, green_channel, blue_channel, red_edge, infrared_edge))

In [16]:
#normalized using minmax method because the maximum and minimum of all five channels were in the same range
img = (img - np.min(img)) / (np.max(img) - np.min(img))

In [17]:
#tiling my image. This block of code will create (3358, 128,128,5), where in simple terms 3358 represents number of sub-images
h, w, bands = img.shape
tileSize = 128
h, w  = h//tileSize, w//tileSize

tiled_image = np.zeros((h*w, tileSize, tileSize, 5))
tiled_labels = np.zeros((h*w, tileSize, tileSize, 1))

k = 0
for i in range(h):
    for j in range(w):
        tiled_image[k] = img[i*tileSize : (i+1)*tileSize, j*tileSize : (j+1)*tileSize, :]
        tiled_labels[k] = labels[i*tileSize : (i+1)*tileSize, j*tileSize : (j+1)*tileSize, :]
        #test_tiled_image[k] = img1[i*tileSize : (i+1)*tileSize, j*tileSize : (j+1)*tileSize, :]
        k+=1

In [18]:
#initialzing the inputs
seed = 42
np.random.seed = seed

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 5

In [19]:
#Building the model
#Source - https://www.kaggle.com/code/mbadrismail/unet-datasciencebowl
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
#print(u6.shape)
#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='softmax')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
# IMportant hyperparameter will be Adam learning rate
# Tried: 5e-04, 8e-04
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  736         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 128, 128, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [20]:
#fitting my model and got good accuracy with 5 epochs
results = model.fit(tiled_image, tiled_labels, batch_size=32, epochs=5)

Epoch 1/5
105/105 [==============================] - 637s 6s/step - loss: 0.8007 - accuracy: 0.6325
Epoch 2/5
105/105 [==============================] - 631s 6s/step - loss: 0.6393 - accuracy: 0.7367
Epoch 3/5
105/105 [==============================] - 631s 6s/step - loss: 0.5498 - accuracy: 0.7805
Epoch 4/5
105/105 [==============================] - 633s 6s/step - loss: 0.4357 - accuracy: 0.8322
Epoch 5/5
105/105 [==============================] - 629s 6s/step - loss: 0.3876 - accuracy: 0.8526


In [21]:
#saved the model for prediction in a separate notebook file.
model.save('my_model.h5')